In [52]:
from etl_func.cut_text import Cut_Machine
import datetime
import os

# setting
os.chdir('/Users/alexlo/Desktop/Project/MLEM_Final/rawdata')
bbs = 'bda2022_mid_bbs_2019-2021.csv'
# news19 = 'bda2022_mid_news_2019.csv'
# news20 = 'bda2022_mid_news_2020.csv'
# news21 = 'bda2022_mid_news_2021.csv'
data_time = (datetime.date(2019,1,1), datetime.date(2021,12,31))
company = '聯電'
keywords_list = [company]

# get data
cut_bbs = Cut_Machine(articles_source=bbs, data_time=data_time)
cut_bbs.article_df = cut_bbs.filter_article(keywords=keywords_list, title_times=1, content_times=2)

In [53]:
final_df = cut_bbs.Pool_sep_all_articles()
final_df.to_csv(company + '.csv', index=False)

+---------------------------------------------------------------------+
  Welcome to MONPA: Multi-Objective NER POS Annotator for Chinese
+---------------------------------------------------------------------++---------------------------------------------------------------------+
  Welcome to MONPA: Multi-Objective NER POS Annotator for Chinese
+---------------------------------------------------------------------+

+---------------------------------------------------------------------+
  Welcome to MONPA: Multi-Objective NER POS Annotator for Chinese
+---------------------------------------------------------------------+
+---------------------------------------------------------------------+
  Welcome to MONPA: Multi-Objective NER POS Annotator for Chinese
+---------------------------------------------------------------------+
+---------------------------------------------------------------------+
  Welcome to MONPA: Multi-Objective NER POS Annotator for Chinese
+---------------------

# 2

In [22]:
%reload_ext autoreload
%autoreload 2

In [54]:
import os
from datetime import date
from etl_func.etl_data import *
from df_func.make_XY import *

rawdata_path = '/Users/alexlo/Desktop/Project/MLEM_Final/rawdata'
os.chdir(rawdata_path)
keywords = ['聯電']
keywords_times_titles = 1
keywords_times_content = 2
article_df = pd.read_csv('bda2022_mid_bbs_2019-2021.csv')
article_df = transform_article_df(article_df, keywords, keywords_times_titles, keywords_times_content)
with open('stopwords_zh.txt', 'r') as file:
    stop_words = file.read().splitlines()
print('finish loading article')

workdata_path = '/Users/alexlo/Desktop/Project/MLEM_Final/workdata'
os.chdir(workdata_path)
all_words = pd.read_csv('/Users/alexlo/Desktop/Project/MLEM_Final/workdata/聯電.csv')
stock_df = pd.read_parquet('聯電.parquet').astype({'Date':'datetime64[ns]'})
stock_df = transform_stock_df(stock_df, D=2, cutoff=3)
print('finish loading stock')

data_time = tuple([date(2019,1,1), date(2021,12,31)])

finish loading article
finish loading stock


In [55]:
words_matrix = Words_Matrix(all_words, stop_words, article_df, stock_df, data_time)
X = words_matrix.feature_X_byChi2(k=50)
Y = words_matrix.Y_matrix

In [56]:
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier
# from sklearn.linear_model import RidgeClassifier
# from sklearn.tree import DecisionTreeClassifier
kNNclf = KNeighborsClassifier()
kNNscores = cross_val_score(kNNclf, X, Y, cv = 5)

In [57]:
kNNscores

array([nan, nan, nan, nan, nan])